# Project Idea 2:
- A good loan (from the prospective of an investor) pays 
the interests and fractional principals on time and terminate at loan maturity.
- An investor often lose money when a loan goes into default, settlement,
or 'written off' (called **charged off** in this data set).
- Build a supervised model to make **multi-label** prediction on 3 dimensions
"charged off + default", "settlement involved", "hardship".
- This can be used either by **Lending Club** itself or a third-party investing firm
for loan-grade design or accurated portfolio selection.
- Depending on the scope of your project, you may 
    - tackle a single label prediction.
    - restrict to the pooled models.
    - focus on the time seris models
- This is a **multi-label** binary imbalance classification task.
- If you train a **pooled** model, you have to deal with $2M+$ samples, often too
large for a typical ML algorithm to handle.

- Try several imbalance classification techniques and evaluate their performance.

- Based on your business, discuss the negative impacts of type I (false
positive), type II (false negative) errors in your prediction.

- If you decide to train a time series model, make sure that you have some
basic background on performing hyper-parameter tuning in the time series context.

- **MUST**: A defaulted loan with a loan amount $\$1000$ has a totally different 
impact to the final profit than a defaulted $\$50000$ loan. 
   - Discuss whether the **classroom-taught** machine learning techniques 
    addresses these issues. How would you modify the classifier to take into account 
         - your business objectives.
         - the profit and loss focus.

- Can you use **NLP** technique to extract insights on the loan descriptions
which helps your predictive task?
</a><br>
# Structure: 
- <a href="#preprocessing">Preprocessing</a><br>
    - <a href="#before">Application Information before loan issued</a><br>
    - <a href="#label">Lables</a><br>
    - <a href="#missing">Missing Values</a><br>
- <a href="#function">Function</a><br>
- <a href="#ml">Machine Learning</a><br>
    - Unsupervised Machine Learning  
        - <a href="#kmeans">K Means</a><br> 

    - Supervised Machine Learning      
        - <a href="#decision">Decision Tree</a><br>
        - <a href="#rf">Random Forest</a><br>
        - <a href="#svm">SVM</a><br>
        - <a href="#xgboost">XGBoost</a><br>
        - <a href="#logistic">Logistic Regression</a><br>
        - <a href="#naive">Naive Bayes Classifier</a><br>
        - <a href="#neighbor">Nearest Neighbor</a><br>
- <a href="#imbalance">Handling Imbalanced Data</a><br>
    - <a href="#smote">SMOTE</a><br>

In [2]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [ ]:
df_raw_accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

In [ ]:
df_raw_accepted.sample(frac=0.001).to_csv('sample_accepted.csv')

In [3]:
sample_accepted = pd.read_csv('sample_accepted.csv')

In [ ]:
sample_accepted.shape

In [5]:
sample_accepted = sample_accepted.sample(2000)

 <p><a name="preprocessing"></a></p>
 
 ## Preprocessing

In [100]:
df_processed = sample_accepted.copy()

 <p><a name="before"></a></p>
 
 ### Application Information before loan is issued 

In [68]:
# Secondary Applicant

joint_list = ['sec_app_fico_range_low','sec_app_fico_range_high',
               'sec_app_earliest_cr_line','sec_app_inq_last_6mths',
               'sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util',
               'sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths',
               'sec_app_collections_12_mths_ex_med','sec_app_mths_since_last_major_derog',
               'verification_status_joint','revol_bal_joint', 
               'dti_joint', 'application_type','annual_inc_joint']

In [95]:
df_processed.loc[df_processed['application_type'] == 'Joint App',joint_list]

,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,verification_status_joint,revol_bal_joint,dti_joint,application_type,annual_inc_joint
1084,745.0,749.0,Jan-2008,0.0,0.0,10.0,6.4,2.0,9.0,0.0,0.0,NaN,Not Verified,6678.0,14.48,Joint App,74000.0
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Verified,NaN,32.11,Joint App,93000.0
617,670.0,674.0,Sep-1999,2.0,3.0,7.0,82.0,3.0,3.0,0.0,0.0,NaN,Not Verified,25384.0,13.49,Joint App,156000.0
786,625.0,629.0,Oct-2008,0.0,0.0,6.0,69.8,1.0,6.0,0.0,0.0,NaN,Not Verified,16287.0,19.24,Joint App,67000.0
1222,650.0,654.0,Sep-2011,2.0,1.0,8.0,35.5,0.0,10.0,0.0,0.0,NaN,Not Verified,19797.0,17.31,Joint App,97200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,630.0,634.0,Sep-2002,2.0,2.0,8.0,77.9,4.0,4.0,0.0,0.0,40.0,Verified,43796.0,28.13,Joint App,140000.0
498,645.0,649.0,Aug-2009,0.0,1.0,4.0,43.2,1.0,6.0,0.0,0.0,65.0,Verified,778.0,7.95,Joint App,56000.0
888,595.0,599.0,Oct-2006,1.0,0.0,14.0,77.0,4.0,16.0,0.0,0.0,NaN,Not Verified,22073.0,27.16,Joint App,75000.0
1090,675.0,679.0,Nov-2001,1.0,2.0,12.0,85.2,2.0,7.0,0.0,0.0,37.0,Not Verified,54167.0,25.70,Joint App,192000.0


 <p><a name="label"></a></p>
 
 ### Lables

In [49]:
# Multi Labels - "charged off + default", "settlement involved", "hardship" 

label_list = ['label_hardship','label_chargedoff_default','label_settlement', 'loan_status' ]
chargedoff_default_list = ['Charged Off','Default', 'Does not meet the credit policy. Status:Charged Off']
# need to based on whole dataset 

df_processed['label_hardship'] = df_processed['hardship_status'].apply(lambda x: 0 if x  is np.nan else 1)
df_processed['label_chargedoff_default'] = df_processed['loan_status'].apply(lambda x: 1 if x in chargedoff_default_list else 0)
df_processed['label_settlement'] = df_processed['settlement_status'].apply(lambda x: 0 if x  is np.nan else 1)

In [105]:
# Hardship Loans 

hardship_list = ['hardship_flag',
     'hardship_type',
     'hardship_reason',
     'hardship_status',
     'deferral_term',
     'hardship_amount',
     'hardship_start_date',
     'hardship_end_date',
     'payment_plan_start_date',
     'hardship_length',
     'hardship_dpd',
     'hardship_loan_status',
     'orig_projected_additional_accrued_interest',
     'hardship_payoff_balance_amount',
     'hardship_last_payment_amount',
     'label_hardship'
                ]

In [30]:
df_processed.loan_status.unique()

array(['Current', 'Fully Paid', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [104]:
df_processed[hardship_list][df_processed['hardship_type'].notnull()]

,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
1057,N,INTEREST ONLY-3 MONTHS DEFERRAL,REDUCED_HOURS,BROKEN,3.0,137.38,Aug-2018,Oct-2018,Aug-2018,3.0,25.0,Late (16-30 days),NaN,15311.53,171.01
492,N,INTEREST ONLY-3 MONTHS DEFERRAL,INCOME_CURTAILMENT,BROKEN,3.0,93.56,Jan-2019,Jan-2019,Jan-2019,3.0,20.0,Late (16-30 days),NaN,9313.45,117.98
1567,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,BROKEN,3.0,112.33,Sep-2017,Oct-2017,Sep-2017,3.0,0.0,Current,NaN,7974.99,1.36
2157,N,INTEREST ONLY-3 MONTHS DEFERRAL,MEDICAL,COMPLETED,3.0,17.74,Jul-2017,Sep-2017,Jul-2017,3.0,20.0,Late (16-30 days),53.22,1296.99,22.75
1166,N,INTEREST ONLY-3 MONTHS DEFERRAL,EXCESSIVE_OBLIGATIONS,BROKEN,3.0,186.47,Aug-2018,Nov-2018,Sep-2018,3.0,11.0,In Grace Period,NaN,15696.84,235.24
214,N,INTEREST ONLY-3 MONTHS DEFERRAL,MEDICAL,COMPLETED,3.0,272.13,Jul-2018,Oct-2018,Aug-2018,3.0,20.0,Late (16-30 days),816.39,22180.82,501.71
1861,N,INTEREST ONLY-3 MONTHS DEFERRAL,EXCESSIVE_OBLIGATIONS,COMPLETED,3.0,233.03,May-2018,Aug-2018,May-2018,3.0,21.0,Late (16-30 days),699.09,11048.50,280.87
680,N,INTEREST ONLY-3 MONTHS DEFERRAL,UNEMPLOYMENT,COMPLETED,3.0,279.08,Jul-2018,Oct-2018,Aug-2018,3.0,9.0,In Grace Period,837.24,25005.03,65.15
277,N,INTEREST ONLY-3 MONTHS DEFERRAL,NATURAL_DISASTER,COMPLETED,3.0,152.74,Sep-2017,Dec-2017,Sep-2017,3.0,0.0,Current,458.22,11563.74,291.43


In [42]:
# Settlement Loans 

settlement_list = ['debt_settlement_flag',
     'debt_settlement_flag_date',
     'settlement_status',
     'settlement_date',
     'settlement_amount',
     'settlement_percentage',
     'settlement_term']

In [101]:
df_processed[settlement_list][df_processed['settlement_status'].notnull()]

,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
57,Y,Aug-2018,ACTIVE,Aug-2018,1389.00,60.00,12.0
1215,Y,Oct-2018,BROKEN,Nov-2017,3378.24,50.00,12.0
2062,Y,Oct-2018,ACTIVE,Oct-2018,8601.00,50.00,18.0
727,Y,Feb-2018,ACTIVE,Feb-2018,3769.00,45.00,14.0
1057,Y,Oct-2018,ACTIVE,Oct-2018,6900.00,45.00,17.0
846,Y,Jan-2018,COMPLETE,May-2017,2233.75,45.00,6.0
548,Y,Oct-2018,COMPLETE,Oct-2017,2246.00,50.00,12.0
598,Y,Apr-2017,COMPLETE,Mar-2017,1463.51,45.00,1.0
1126,Y,Mar-2018,ACTIVE,Mar-2018,4127.00,50.00,18.0
639,Y,Aug-2018,ACTIVE,Aug-2018,2986.00,40.00,12.0


In [65]:
df_processed[label_list].sample(10)

,label_hardship,label_chargedoff_default,label_settlement,loan_status
77,0,0,0,Fully Paid
974,0,0,0,Fully Paid
216,0,0,0,Fully Paid
53,0,0,0,Fully Paid
1214,0,0,0,Fully Paid
202,0,0,0,Current
1532,0,1,0,Charged Off
1229,0,0,0,Fully Paid
722,0,0,0,Current
518,0,0,0,Fully Paid


 <p><a name="missing"></a></p>
    
### Missing Values 

In [98]:
total = df_processed.isnull().sum().sort_values(ascending=False)
percent = (df_processed.isnull().sum()/df_processed.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(100)

,Total,Percent
member_id,2000,1.0000
orig_projected_additional_accrued_interest,1995,0.9975
payment_plan_start_date,1991,0.9955
hardship_start_date,1991,0.9955
hardship_type,1991,0.9955
hardship_reason,1991,0.9955
hardship_status,1991,0.9955
deferral_term,1991,0.9955
hardship_last_payment_amount,1991,0.9955
hardship_payoff_balance_amount,1991,0.9955


In [10]:
# Drop irrelavant columns 
drop_list = ['Unnamed: 0','id','member_id','funded_amnt','url','desc','title']

drop_for_grade_list = ['funded_amnt_inv','int_rate','installment','issue_d','loan_status','pymnt_plan','out_prncp','out_prncp_inv']

df_processed = df_processed.drop(drop_list, axis=1)
df_processed = df_processed.drop(drop_for_grade_list, axis=1)

# Convert categorical to numerical 
df_processed['term'] = df_processed['term'].apply(lambda x: int(x.split()[0]))
df_processed['emp_length'] = df_processed['emp_length'].str.extract('(\d+)') 
#10 means more than 10 years 

# Convert to Datetime
df_processed['earliest_cr_line'] = pd.to_datetime(df_processed['earliest_cr_line'])

# Missing Values 

df_processed.mths_since_last_record = df_processed.mths_since_last_record.fillna(0)
df_processed.mths_since_last_delinq = df_processed.mths_since_last_delinq.fillna(0)

df_processed.emp_title = df_processed.emp_title.fillna('None')
df_processed.emp_length = df_processed.emp_length.fillna(0)

df_processed.revol_util = df_processed.revol_util.fillna(0)

df_processed.dti = df_processed.dti.fillna(df_processed.revol_bal / df_processed.annual_inc)